In [7]:
%load_ext autoreload
%autoreload 2

import pandas as pd

paths = [
    "data/training.jsonl",
    "data/TAR_data.jsonl",
    "data/sysrev_conv.jsonl",
]
dataset = []
for path in paths:
    df = pd.read_json(path, lines=True)
    dataset.append(df)

dataset = pd.concat(dataset)
dataset = dataset[dataset["nl_query"] != ""]
dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,nl_query,bool_query,source,quality,pmid,id,link_to_review,search_name,Date_from,Date_to,seed_studies,included_studies,edited_search
0,Topical antibiotic prophylaxis to reduce respi...,((critical care OR intensive care OR burn uni...,raw-jsonl,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Extra-abdominal versus intra-abdominal repair ...,(exteriorization or exteriorisation or extra-...,raw-jsonl,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Antiepileptic drugs for preventing seizures fo...,((phenytoin* OR carbamazepine* OR pentobarbit...,raw-jsonl,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Anthelmintics for people with neurocysticercosis,"((neurocysticerc* OR ""brain cysticerc*"" OR ""c...",raw-jsonl,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Repeated lumbar or ventricular punctures in ne...,"((infant, newborn[MeSH] OR newborn OR neonate...",raw-jsonl,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,A catalogue of tools and variables from crisis...,"(""Resource Allocation""[Mesh] OR Resource[tiab]...",sysrev,NaN,NaN,13.0,https://pubmed.ncbi.nlm.nih.gov/34438132/,ICU ventilator allocation,01/01/1946,14/05/2020,"[21269458, 30574216, 17116904, 25144591, 25144...","[32289312, 17116904, 18460506, 32660660, 18388...","(""Resource Allocation""[Mesh] OR Resource[tiab]..."
36,Challenges and facilitators in delivering opti...,(Aged[Mesh] OR Elderly[tiab] OR Older[tiab] OR...,sysrev,NaN,NaN,66.0,https://pubmed.ncbi.nlm.nih.gov/33713331/,Possible bias in reporting consumer perspectiv...,01/01/1946,13/03/2021,"[30446268, 27846832, 30868725, 30614161, 26124...","[25390467, 27815169, 25892177, 23453467, 21631...",(Aged[Mesh] OR Elderly[tiab] OR Older[tiab] OR...
37,Impact of COVID-19 pandemic on utilisation of ...,"(""COVID-19""[Supplementary Concept] OR “COVID-1...",sysrev,NaN,NaN,14.0,https://pubmed.ncbi.nlm.nih.gov/33727273/,Impact of Covid-19 on hospital systems (admiss...,01/01/1946,10/08/2020,"[32596921, 32437340, 32679205, 32511563, 32525...","[32482087, 33501967, 32713658, 32590109, 32501...","(""COVID-19""[Supplementary Concept] OR ""COVID-1..."
38,Downsides of face masks and possible mitigatio...,"((""Masks/adverse effects""[MAJR]) OR ((Masks[Me...",sysrev,NaN,NaN,1.0,https://pubmed.ncbi.nlm.nih.gov/33619199/,Adverse effects of masks,01/01/1946,19/05/2020,"[5802921, 18500410, 16671853, 3631620, 3235285...","[22295066, 20836797, 20036443, 25336079, 18295...","(""Telemedicine""[Mesh] OR ""Videoconferencing""[M..."


In [6]:
N = 10000
df = dataset.sample(min(N, dataset.shape[0])).reset_index(drop=True)
df

,nl_query,bool_query,source,quality,pmid,id,link_to_review,search_name,Date_from,Date_to,seed_studies,included_studies,edited_search
0,,"""Journal of the American College of Cardiology...",pubmed-query,0.1,15734619.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Increased Risk of Lymphoma in Men or the Elder...,"burketts[All Fields] AND (""lymphoma""[MeSH Term...",pubmed-query,0.1,34527205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Updates in Hormone Replacement Therapy for Sur...,"""Hormone Replacement Therapy""[MeSH] AND ""Genit...",pubmed-query,0.1,40042741.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Beta-blocker plus nitrates for secondary preve...,(beta-blocker* OR 'adrenergic beta antagonist...,raw-jsonl,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"BACTERIAL PROFILE, ANTIMICROBIAL RESISTANCE, A...","(""Cefuroxime""[MeSH] OR ""cefuroxime axetil""[Sub...",pubmed-query,0.1,37991965.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,,"(""myofascial pain syndromes""[TIAB] NOT Medline...",pubmed-query,0.1,40630147.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,,"""Apoptosis : an international journal on progr...",pubmed-query,0.1,16151643.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,,(C1QTNF5[Title/Abstract] OR LORD[Title/Abstrac...,pubmed-query,0.1,40487283.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,Estrogen treatment improves spatial learning i...,Heikkinen T [AU] AND Kalesnykas G [AU] AND 200...,pubmed-query,0.1,15081593.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
import nltk
import numpy as np

nltk.download("words")
words = nltk.corpus.words.words()
words = np.random.choice(words, 10000).tolist()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Simon\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [9]:
import torch
from boolrank import DualSiglip2Model

model = DualSiglip2Model('BAAI/bge-small-en-v1.5')
model.load(r"models\clip\bge-small-en-v1.5\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
# model = DualSiglip2Model('dmis-lab/biobert-v1.1')
# model.load(r"models\clip\biobert-v1.1\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-14110\model.safetensors")

embeddings = model.encode_bool(df["bool_query"].tolist(), batch_size=200).detach().cpu().numpy()
# embeddings = model.encode_text(words, batch_size=200).detach().cpu().numpy()
torch.cuda.empty_cache()

In [11]:
import umap

um = umap.UMAP(n_neighbors=15, n_components=3)
trans = um.fit_transform(embeddings)

x = trans[:,0]
y = trans[:,1]
z = trans[:,2]
df["x"] = x
df["y"] = y

def cutoff(n): return lambda x: x if len(x) < n else x[:n] + "..."
cut = 60
df["nl"] = df["nl_query"].map(cutoff(cut))
df["bool"] = df["bool_query"].map(cutoff(cut))

In [15]:
query = dataset.iloc[10]["nl_query"]
# query = "cancer"
query_emb = model.encode_text(query).detach().cpu().numpy()
query

'Nitric oxide for respiratory failure in infants born at or near term'

In [16]:
similarity = model.get_similarities(embeddings, query_emb).numpy()
df["sim"] = similarity
df["sim"].values

array([0.42253995, 0.2588103 , 0.15726179, ..., 0.5463351 , 0.19548303,
       0.33639807], shape=(10000,), dtype=float32)

In [17]:
top_n = (-similarity).argsort()[:100]
mask = np.zeros_like(similarity)
# bool_mask = mask + 1
# bool_mask[top_n] = 0
mask[top_n] = 0.9
mask += 0.01

In [18]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.x,
        y=df.y,
        mode='markers',
        marker=dict(
            opacity=mask,
            ),
        fillpattern=dict(
            fillmode="overlay"
        ),
        hovertext=df["bool"]
    )
)

fig.update_layout(width=1000, height=700)
fig.show()